In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages
import statsmodels.stats.multitest as smm
import numpy as np
#from bioinfokit import analys, visuz
from matplotlib.patches import Circle
from matplotlib.patheffects import withStroke
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
import matplotlib

color_palette = sns.color_palette("colorblind")

sns.set_theme(palette=color_palette, font="Arial", font_scale=1.0, style="white")

import os


In [2]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 25)
#pd.set_option('display.max_colwidth', 300)
#pd.set_option('format.precision', 2)




In [4]:
# load results table from R DESeq analysis (gene-level, LFC shrinkage)
results_genes=pd.read_csv("gene_resAnno_only_female.csv")  
results_genes.rename(columns={"Unnamed: 0": "GENEID"}, inplace = True)

results_genes#.dropna(axis=0, subset=["SYMBOL"])
results_genes #all GENEIDs are unique

,GENEID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,GENENAME,SYMBOL
0,ENSMUSG00000000001,6486.858231,0.319575,0.084545,3.779926,0.000157,0.004735,Gnai3,Gnai3
1,ENSMUSG00000000003,0.000000,NaN,NaN,NaN,NaN,NaN,Pbsn,Pbsn
2,ENSMUSG00000000028,179.606989,0.496340,0.209930,2.364310,0.018064,0.128084,Cdc45,Cdc45
3,ENSMUSG00000000037,33.474291,-0.258078,0.615465,-0.419322,0.674981,NaN,Scml2,Scml2
4,ENSMUSG00000000049,2.997353,2.376936,1.844183,1.288883,0.197439,NaN,Apoh,Apoh
...,...,...,...,...,...,...,...,...,...
35677,ENSMUSG00000118670,2.865875,-0.266241,1.296495,-0.205355,0.837295,NaN,NaN,NaN
35678,ENSMUSG00000118671,251.083854,0.196333,0.251715,0.779979,0.435403,0.727331,EPPK1,EPPK1
35679,ENSMUSG00000118672,0.000000,NaN,NaN,NaN,NaN,NaN,Muc4,Muc4
35680,ENSMUSG00001074846,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#formatting
data=results_genes
data["padj_rounded"] = data["padj"].map('{:.1e}'.format)
data["GENENAME_padj"] = data["GENENAME"] + "\np=" + data["padj_rounded"].astype(str) +""
data["-log10_padj"] = -np.log10(data["padj"])

In [6]:
padj_thr = 0.05
lfc_thr = 1




data_all = data.dropna(subset=["log2FoldChange", "padj", "GENENAME"])
data_all_up = data_all[(data_all["log2FoldChange"] > lfc_thr) & (data_all["padj"] < 0.05)]
data_all_down = data_all[(data_all["log2FoldChange"] < -lfc_thr) & (data_all["padj"] < 0.05)]

In [7]:
data_DE_all_sortL2FC = pd.concat([data_all_up.sort_values(by=["log2FoldChange"], ascending=False),
           data_all_down.sort_values(by=["log2FoldChange"], ascending=False)])

print(len(data_DE_all_sortL2FC))
data_DE_all_sortL2FC

52


,GENEID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,GENENAME,SYMBOL,padj_rounded,GENENAME_padj,-log10_padj
7672,ENSMUSG00000030228,295.803410,2.157526,0.532874,4.048851,5.146978e-05,2.093235e-03,Pik3c2g,Pik3c2g,2.1e-03,Pik3c2g\np=2.1e-03,2.679182
9074,ENSMUSG00000032776,64.235516,2.091380,0.549920,3.803064,1.429173e-04,4.376582e-03,Mctp2,Mctp2,4.4e-03,Mctp2\np=4.4e-03,2.358865
2959,ENSMUSG00000021214,5813.879494,2.008155,0.499583,4.019661,5.828192e-05,2.259628e-03,Akr1c18,Akr1c18,2.3e-03,Akr1c18\np=2.3e-03,2.645963
3388,ENSMUSG00000021983,41.360301,1.994092,0.666320,2.992696,2.765250e-03,3.721639e-02,Atp8a2,Atp8a2,3.7e-02,Atp8a2\np=3.7e-02,1.429266
3089,ENSMUSG00000021453,248.881327,1.799815,0.297282,6.054226,1.410942e-09,4.259029e-07,Gadd45g,Gadd45g,4.3e-07,Gadd45g\np=4.3e-07,6.370689
15194,ENSMUSG00000053007,4644.332243,1.637491,0.175025,9.355749,8.300911e-21,3.507965e-17,Creb5,Creb5,3.5e-17,Creb5\np=3.5e-17,16.454945
8565,ENSMUSG00000031872,158.769300,1.517528,0.315108,4.815893,1.465432e-06,1.317641e-04,Bean1,Bean1,1.3e-04,Bean1\np=1.3e-04,3.880203
33028,ENSMUSG00000113395,61.554082,1.482490,0.354932,4.176826,2.956047e-05,1.348085e-03,Gm48365,Gm48365,1.3e-03,Gm48365\np=1.3e-03,2.870283
4141,ENSMUSG00000023868,1014.548081,1.455722,0.402095,3.620343,2.942130e-04,7.627878e-03,Pde10a,Pde10a,7.6e-03,Pde10a\np=7.6e-03,2.117596
6334,ENSMUSG00000027894,52.771663,1.453162,0.466460,3.115299,1.837582e-03,2.786707e-02,Slc6a17,Slc6a17,2.8e-02,Slc6a17\np=2.8e-02,1.554909


In [8]:
data_DE_all_sortL2FC.sort_values(by=["log2FoldChange"], ascending=True)[["GENENAME", 
                                                                         "log2FoldChange", 
                                                                         "padj_rounded"]]\
.round(3)\
.to_excel("data_l2fc_p_DEG_only_female_231212.xlsx")

In [10]:
genes_of_interest = ["Cacna1a",
                    "Cacna1b",
                    "Cacna1c",
                    "Cacna1d",
                    "Cacna1e",
                    "Cacna1f",
                    "Cacna1g",
                    "Cacna1h",
                    "Cacna1i",
                    "Cacna2d1",
                    "Cacna2d2",
                    "Cacnb1",
                    "Cacnb2",
                    "Cacnb3",
                    "Cacnb4"
                    ]

In [11]:
data_goi = data[data["GENENAME"].isin(genes_of_interest)]
data_goi.sort_values(by=["GENENAME"], ascending=True)

,GENEID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,GENENAME,SYMBOL,padj_rounded,GENENAME_padj,-log10_padj
9696,ENSMUSG00000034656,59.315295,-0.704451,0.455928,-1.545093,1.223237e-01,3.976461e-01,Cacna1a,Cacna1a,4.0e-01,Cacna1a\np=4.0e-01,0.400503
673,ENSMUSG00000004113,45.643195,0.799207,0.406709,1.965056,4.940770e-02,2.316534e-01,Cacna1b,Cacna1b,2.3e-01,Cacna1b\np=2.3e-01,0.635161
14851,ENSMUSG00000051331,3815.011557,0.200981,0.121228,1.657872,9.734339e-02,3.479333e-01,Cacna1c,Cacna1c,3.5e-01,Cacna1c\np=3.5e-01,0.458504
1674,ENSMUSG00000015968,256.024213,0.620129,0.188000,3.298560,9.718218e-04,1.817157e-02,Cacna1d,Cacna1d,1.8e-02,Cacna1d\np=1.8e-02,1.740608
672,ENSMUSG00000004110,9.562112,0.596897,0.856821,0.696641,4.860274e-01,NaN,Cacna1e,Cacna1e,nan,Cacna1e\np=nan,NaN
8133,ENSMUSG00000031142,2.967424,-1.218559,1.589666,-0.766551,4.433487e-01,NaN,Cacna1f,Cacna1f,nan,Cacna1f\np=nan,NaN
2768,ENSMUSG00000020866,23.656461,-0.435087,0.520216,-0.836359,4.029532e-01,NaN,Cacna1g,Cacna1g,nan,Cacna1g\np=nan,NaN
4278,ENSMUSG00000024112,2000.688488,-1.682962,0.118760,-14.171075,1.383663e-45,8.771041e-42,Cacna1h,Cacna1h,8.8e-42,Cacna1h\np=8.8e-42,41.056949
3640,ENSMUSG00000022416,25.848093,0.204966,0.638909,0.320806,7.483576e-01,NaN,Cacna1i,Cacna1i,nan,Cacna1i\np=nan,NaN
11696,ENSMUSG00000040118,7865.999032,0.480032,0.113218,4.239900,2.236196e-05,1.065808e-03,Cacna2d1,Cacna2d1,1.1e-03,Cacna2d1\np=1.1e-03,2.972321


In [12]:
kegg_list = pd.read_csv("Kegg_list_mmu.csv", 
                        usecols=['GeneID', 'PathwayID','Symbol', 'ENSEMBL', "Genename"])
kegg_aldo = kegg_list[kegg_list["PathwayID"]=="path:mmu04925"]

data_aldo_genes = kegg_aldo.merge(results_genes, how="left", left_on='ENSEMBL', right_on='GENEID', validate="1:1")
print(len(data_aldo_genes.sort_values(by=["log2FoldChange"], ascending=True)))
data_aldo_genes.sort_values(by=["log2FoldChange"], ascending=True)

102


,GeneID,PathwayID,Symbol,ENSEMBL,Genename,GENEID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,GENENAME,SYMBOL,padj_rounded,GENENAME_padj,-log10_padj
57,18227,path:mmu04925,Nr4a2,ENSMUSG00000026826,"nuclear receptor subfamily 4, group A, member 2",ENSMUSG00000026826,901.539706,-2.363813,1.133981,-2.084526,3.711236e-02,1.988112e-01,Nr4a2,Nr4a2,2.0e-01,Nr4a2\np=2.0e-01,0.701559
93,58226,path:mmu04925,Cacna1h,ENSMUSG00000024112,"calcium channel, voltage-dependent, T type, al...",ENSMUSG00000024112,2000.688488,-1.682962,0.118760,-14.171075,1.383663e-45,8.771041e-42,Cacna1h,Cacna1h,8.8e-42,Cacna1h\np=8.8e-42,41.056949
41,15370,path:mmu04925,Nr4a1,ENSMUSG00000023034,"nuclear receptor subfamily 4, group A, member 1",ENSMUSG00000023034,5313.643828,-1.260267,1.114075,-1.131222,2.579616e-01,5.753217e-01,Nr4a1,Nr4a1,5.8e-01,Nr4a1\np=5.8e-01,0.240089
92,54652,path:mmu04925,Cacna1f,ENSMUSG00000031142,"calcium channel, voltage-dependent, alpha 1F s...",ENSMUSG00000031142,2.967424,-1.218559,1.589666,-0.766551,4.433487e-01,NaN,Cacna1f,Cacna1f,nan,Cacna1f\np=nan,NaN
85,269060,path:mmu04925,Dagla,ENSMUSG00000035735,"diacylglycerol lipase, alpha",ENSMUSG00000035735,65.765206,-1.099050,0.302819,-3.629395,2.840865e-04,7.441422e-03,Dagla,Dagla,7.4e-03,Dagla\np=7.4e-03,2.128344
25,12292,path:mmu04925,Cacna1s,ENSMUSG00000026407,"calcium channel, voltage-dependent, L type, al...",ENSMUSG00000026407,3.606254,-1.084701,1.213527,-0.893842,3.714064e-01,NaN,Cacna1s,Cacna1s,nan,Cacna1s\np=nan,NaN
62,18752,path:mmu04925,Prkcg,ENSMUSG00000078816,"protein kinase C, gamma",ENSMUSG00000078816,5.168329,-1.048316,1.192000,-0.879460,3.791520e-01,NaN,Prkcg,Prkcg,nan,Prkcg\np=nan,NaN
51,16521,path:mmu04925,Kcnj5,ENSMUSG00000032034,"potassium inwardly-rectifying channel, subfami...",ENSMUSG00000032034,45.683308,-0.867625,0.382738,-2.266889,2.339703e-02,1.515726e-01,Kcnj5,Kcnj5,1.5e-01,Kcnj5\np=1.5e-01,0.819379
94,67821,path:mmu04925,Atp1b4,ENSMUSG00000016327,"ATPase, (Na+)/K+ transporting, beta 4 polypeptide",ENSMUSG00000016327,0.162192,-0.782954,4.080473,-0.191878,8.478375e-01,NaN,Atp1b4,Atp1b4,nan,Atp1b4\np=nan,NaN
69,18976,path:mmu04925,Pomc,ENSMUSG00000020660,pro-opiomelanocortin-alpha,ENSMUSG00000020660,5.121558,-0.696873,0.994334,-0.700844,4.834003e-01,NaN,Pomc,Pomc,nan,Pomc\np=nan,NaN


In [14]:
data_aldo_genes

,GeneID,PathwayID,Symbol,ENSEMBL,Genename,GENEID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,GENENAME,SYMBOL,padj_rounded,GENENAME_padj,-log10_padj
0,100043456,path:mmu04925,Hsd3b8,ENSMUSG00000095388,"hydroxy-delta-5-steroid dehydrogenase, 3 beta-...",ENSMUSG00000095388,0.000000,NaN,NaN,NaN,NaN,NaN,Gm10681,Gm10681,nan,Gm10681\np=nan,NaN
1,100043461,path:mmu04925,Hsd3b9,ENSMUSG00000090817,"hydroxy-delta-5-steroid dehydrogenase, 3 beta-...",ENSMUSG00000090817,0.000000,NaN,NaN,NaN,NaN,NaN,Gm4450,Gm4450,nan,Gm4450\np=nan,NaN
2,101540,path:mmu04925,Prkd2,ENSMUSG00000041187,protein kinase D2,ENSMUSG00000041187,239.025892,0.001991,0.212465,0.009370,9.925238e-01,9.980344e-01,Prkd2,Prkd2,1.0e+00,Prkd2\np=1.0e+00,0.000855
3,104110,path:mmu04925,Adcy4,ENSMUSG00000022220,adenylate cyclase 4,ENSMUSG00000022220,157.549892,-0.115343,0.218021,-0.529046,5.967737e-01,8.352464e-01,Adcy4,Adcy4,8.4e-01,Adcy4\np=8.4e-01,0.078185
4,104111,path:mmu04925,Adcy3,ENSMUSG00000020654,adenylate cyclase 3,ENSMUSG00000020654,771.103325,-0.061749,0.113386,-0.544593,5.860331e-01,8.282924e-01,Adcy3,Adcy3,8.3e-01,Adcy3\np=8.3e-01,0.081816
5,108058,path:mmu04925,Camk2d,ENSMUSG00000053819,calcium/calmodulin-dependent protein kinase II...,ENSMUSG00000053819,4659.073264,0.322220,0.086392,3.729727,1.916873e-04,5.473450e-03,Camk2d,Camk2d,5.5e-03,Camk2d\np=5.5e-03,2.261739
6,109305,path:mmu04925,Orai1,ENSMUSG00000049686,ORAI calcium release-activated calcium modulat...,ENSMUSG00000049686,151.411743,-0.257652,0.208723,-1.234422,2.170459e-01,5.308078e-01,Orai1,Orai1,5.3e-01,Orai1\np=5.3e-01,0.275063
7,11512,path:mmu04925,Adcy6,ENSMUSG00000022994,adenylate cyclase 6,ENSMUSG00000022994,962.387890,0.147670,0.113638,1.299482,1.937785e-01,5.004622e-01,Adcy6,Adcy6,5.0e-01,Adcy6\np=5.0e-01,0.300629
8,11513,path:mmu04925,Adcy7,ENSMUSG00000031659,adenylate cyclase 7,ENSMUSG00000031659,690.529756,-0.074040,0.136668,-0.541753,5.879889e-01,8.286306e-01,Adcy7,Adcy7,8.3e-01,Adcy7\np=8.3e-01,0.081639
9,11514,path:mmu04925,Adcy8,ENSMUSG00000022376,adenylate cyclase 8,ENSMUSG00000022376,70.962579,0.682274,0.572679,1.191371,2.335078e-01,5.495474e-01,Adcy8,Adcy8,5.5e-01,Adcy8\np=5.5e-01,0.259995


In [13]:
data_aldo_genes.sort_values(by=["padj", "log2FoldChange"], ascending=True).to_excel("kegg_aldo_mmu04925_genes_only_female_231212.xlsx",
                                                            columns=["Symbol", "log2FoldChange", "padj"])